In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('names.tsv',encoding='cp1251', sep='\t')
data.head()

,name
0,Соус АСТОРИЯ сырный 233г
1,Соус Сэн Сой маринад 500гр
2,Ю253 Соус Calve барбекю латино 230г 1*28
3,соус МИСТЕР РИККО 210г сырный дой-пак (18*1)
4,"Кетчуп ""Махеевъ"" 500г чили пл.пак."


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11530 entries, 0 to 11529
Data columns (total 1 columns):
name    11530 non-null object
dtypes: object(1)
memory usage: 90.2+ KB


Всего 11530 строк.

Сведем все к строчным буквам

In [4]:
data.name = data.name.str.lower()
data.head()

,name
0,соус астория сырный 233г
1,соус сэн сой маринад 500гр
2,ю253 соус calve барбекю латино 230г 1*28
3,соус мистер рикко 210г сырный дой-пак (18*1)
4,"кетчуп ""махеевъ"" 500г чили пл.пак."


разобъем по словам

In [5]:
words = data.name.str.split(' ', expand=True)
words.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,соус,астория,сырный,233г,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,соус,сэн,сой,маринад,500гр,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,ю253,соус,calve,барбекю,латино,230г,1*28,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,соус,мистер,рикко,210г,сырный,дой-пак,(18*1),None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,кетчуп,"""махеевъ""",500г,чили,пл.пак.,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Чисто из любопытства посмотрим самую длинную строку

In [6]:
words.loc[words[34].notnull() == True, :]

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
1234,кетчуп,махеевъ,260гр,русский,с,кусочками,овощей,дой-пак,,,...,,,,,,,@,каталог,с,17.07.2017-06.08.2017


Почему-то используется большое количество пробелов, но в нашем случае это не является ошибкой.

Основную информацию по товару содержат первые два столбца. Именно по ним и производим разметку.
Для выполнения поставленной задачи действуем по следующему алгоритму:

- Разметим самые очевидные категории: 'майонез', 'кетчуп', 'паштет', 'оливки'. Они легко устанавливаются по названию. В некоторых случаях, конечно, возможны сокращения, либо информация была представлена без пробелов, по этому используем поиск по маске.
        
- Выбираем и разбираем самые многочисленные категории
  
Т.к. задание тестовое разберем 80% выборки.Более основательное рассмотрение будет уже менее эффективно и более трудозатратно.
Безусловно можно использовать специализированные библиотеки, но т.к. предполагается дальнейшая передача материала для специалистов по ML, скорее, это лишнее.


In [7]:
words[0].value_counts()[:5]

майонез     2528
соус        1989
кетчуп      1701
приправа     425
горчица      309
Name: 0, dtype: int64

In [8]:
words[1].value_counts()[:5]

махеев        757
махеевъ       555
слобода       442
майонезный    355
ряба          311
Name: 1, dtype: int64

In [9]:
fts = ['майонез', 'кетчуп', 'паштет', 'оливки']

In [10]:
for i in range(len(fts)):
    words.loc[words[0].str.contains(fts[i], na=False), 'category'] = fts[i]
    words.loc[words[1].str.contains(fts[i], na=False), 'category'] = fts[i]

In [11]:
def persents(words):
    return (11530 - words.category.isnull().sum()) / 11530 * 100

In [12]:
print(f'выборка размечена на {persents(words):.2f}%')

выборка размечена на 46.75%


In [13]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

соус        1617
приправа     425
горчица      309
аджика       306
соль         257
Name: 0, dtype: int64

In [14]:
words.loc[words[0] == 'соус', 1].value_counts()[:5]

майонезный    339
кальве        221
кинто         186
сэн           132
соевый        124
Name: 1, dtype: int64

In [15]:
words.loc[words[0].str.contains('соус', na=False), 'category'] = 'соусы/заправки'
words.loc[words[1].str.contains('соус', na=False), 'category'] = 'соусы/заправки'

In [16]:
print(f'выборка размечена на {persents(words):.2f}%')

выборка размечена на 63.01%


In [17]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

приправа    425
горчица     309
аджика      306
соль        257
уксус       196
Name: 0, dtype: int64

In [18]:
words.loc[words[0] == 'приправа', 1].value_counts()[:5]

кнорр     85
для       60
мастер    35
22        34
магги     30
Name: 1, dtype: int64

In [19]:
words.loc[words[0].str.contains('приправ', na=False), 'category'] = 'приправы/консерванты/экстракты (наборы)'

In [20]:
words.loc[words[0].str.contains('уксус', na=False), 'category'] = 'уксусы/кулинарное вино'
words.loc[words[0].str.contains('аджик', na=False), 'category'] = 'приправы/консерванты/экстракты'
words.loc[words[0].str.contains('горчиц', na=False), 'category'] = 'приправы/консерванты/экстракты'
words.loc[words[0].str.contains('соль', na=False), 'category'] = 'приправы/консерванты/экстракты (другие)'

In [21]:
print(f'выборка размечена на {persents(words):.2f}%')

выборка размечена на 76.70%


In [22]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

паста       172
икра        113
заправка    109
томатная    101
маринад      95
Name: 0, dtype: int64

In [23]:
words.loc[words.category.isnull() == True, 1].value_counts()[:5]

томатная    167
паста       125
махеев       86
на           67
из           61
Name: 1, dtype: int64

In [24]:
spred = ['паста', 'икра']
for j in range(len(spred)):
    words.loc[words[0].str.contains(spred[j], na=False), 'category'] = 'спред'
    words.loc[words[1].str.contains(spred[j], na=False), 'category'] = 'спред'

In [25]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

заправка    109
маринад      95
огурцы       85
перец        84
смесь        81
Name: 0, dtype: int64

In [26]:
words.loc[words[0].str.contains('заправк', na=False), 'category'] = 'заправки/соусы (наборы)'

In [27]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

маринад    95
огурцы     85
перец      84
смесь      81
мастер     55
Name: 0, dtype: int64

In [28]:
print(f'выборка размечена на {persents(words):.2f}%')

выборка размечена на 81.87%


In [29]:
words.loc[words[0] == 'маринад', 'category'] = 'приправы/консерванты/экстракты (наборы)'
words.loc[words[1] == 'маринад', 'category'] = 'приправы/консерванты/экстракты (наборы)'

In [30]:
print(f'выборка размечена на {persents(words):.2f}%')

выборка размечена на 82.78%


In [31]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

огурцы    85
перец     84
смесь     81
мастер    55
грибы     55
Name: 0, dtype: int64

In [32]:
words.loc[(words[0] == 'огурцы') & (words[1] == 'резанные') , :]

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,category
667,огурцы,резанные,500,г.,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN


Огурцы резанные - это явная ошибка, т.к. не относится к консервантам/солениям и т.д.

In [33]:
words.loc[words[0] == 'огурцы', 'category'] = 'соленье'
words.loc[words[1] == 'огурцы', 'category'] = 'соленье'
words.loc[(words[0] == 'огурцы') & (words[1] == 'резанные') , 'category'] = 'trash'

In [34]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

перец     84
смесь     81
мастер    55
грибы     55
кнорр     49
Name: 0, dtype: int64

In [35]:
words.loc[words[0] == 'перец', 1].value_counts()

черный      38
красный     13
мастер       5
пять         5
чёрный       4
горошек      3
царск        2
все          2
авс          2
"жар         2
душистый     2
сладкий      1
чили         1
капо         1
черн.        1
черн         1
чернный      1
Name: 1, dtype: int64

сладкий перец - это явно овощи, т.е. тоже ошибка.

In [36]:
words.loc[words[0] == 'перец', 'category'] = 'травы/специи/экстракты'
words.loc[(words[0] == 'перец') & (words[1] == 'сладкий'), 'category'] = 'trash'

In [37]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

смесь     81
грибы     55
мастер    55
кнорр     49
м-з       44
Name: 0, dtype: int64

In [38]:
words.loc[words[0] == 'смесь', 1].value_counts()

knorr              21
кнорр              16
магги              13
на                  8
сухая               6
сух.                5
кнор                4
maggi               3
"магги"             2
для                 1
д/приготовление     1
д/сочной            1
Name: 1, dtype: int64

In [39]:
words.loc[words[0] == 'смесь', 'category'] = 'приправы/консерванты/экстракты (наборы)'

In [40]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

мастер    55
грибы     55
кнорр     49
м-з       44
томаты    40
Name: 0, dtype: int64

In [41]:
words.loc[words[0] == 'мастер', 1].value_counts()

дак    54
шеф     1
Name: 1, dtype: int64

In [42]:
words.loc[words[0] == 'мастер', 'category'] = 'приправы/консерванты/экстракты (наборы)'

In [43]:
words.loc[words.category.isnull() == True, 0].value_counts()[:5]

грибы     55
кнорр     49
м-з       44
томаты    40
"жар      37
Name: 0, dtype: int64

In [44]:
words.loc[words[0] == 'грибы', :]

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,category
110,грибы,дары,края,белые,маринованные,550г,ведерко;,2,None,None,...,None,None,None,None,None,None,None,None,None,NaN
164,грибы,барко,580мл,маслята,марин,ст/б,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
190,грибы,опята,маринованные,250г,хакасский,рпс;,5,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
522,грибы,лесная,былина,250г,сб,волнушки,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
561,грибы,главпродукт,опята,марин,580мл,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
582,грибы,мелен,лисички,540г,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
755,грибы,шамп,спело,зрело,мар,сб,0.5кг,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
1586,грибы,принцесса,вкуса,280г,шампиньоны,маринованные,ст/б,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
1821,грибы,310,мл,алтайские,1/12,огородников,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN
2102,грибы,мелен,владимирские,540г,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,NaN


In [45]:
print(f'выборка размечена на {persents(words):.2f}%')

выборка размечена на 85.54%


грибы целые и резанные тоже стоит отнести к ошибкам, но это уже скорее будет инструкцией асессорам.

In [46]:
words.loc[words.category.isnull() == True, 'category'] = 'assessing'


In [47]:
words.loc[words.category.isnull() == True, 'category']

Series([], Name: category, dtype: object)

In [48]:
data['category'] = words['category']
data.head()

,name,category
0,соус астория сырный 233г,соусы/заправки
1,соус сэн сой маринад 500гр,соусы/заправки
2,ю253 соус calve барбекю латино 230г 1*28,соусы/заправки
3,соус мистер рикко 210г сырный дой-пак (18*1),соусы/заправки
4,"кетчуп ""махеевъ"" 500г чили пл.пак.",кетчуп


In [49]:
data.to_csv('result.tsv', sep='\t', encoding='cp1251')

В качестве дополнения:

- Желательно предварительно изучить курс по тавароведению и действующее законодательство по классифицируемым товарам.
- При назначении категории учитывать цель классифицирования. Например, для транспортировки значения будут иметь одни признаки, а при хранении - другие.